![Walmart](https://4.bp.blogspot.com/-US_5wafsHJI/VAo33JnvTZI/AAAAAAAADS4/k4tXNXvuD9k/s1600/Attention%2BWalmart%2BShoppers.png)

## Capstone Presentation
Cindy Villanueva | Alberto Puentes | Heather McMillan | Natasha Rivers

#### August 19, 2021

<hr style="border:3px solid black"> </hr>

# Imports

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy.stats as stats

import math

#visualization
import matplotlib.pyplot as plt
import seaborn as sns

#custom modules
import new_wrangle as w
import model as m

#sklearn preprossing, clustering and modeling libraries
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.model_selection import cross_val_score

#remove warnings
import warnings
warnings.filterwarnings("ignore")

## Color Palette Specs for Visualization

In [2]:
#color palette for visualizations
walmart_palette_c = ['#004c91', '#007dc6', '#78b9e7', '#f47321', '#ffc220', '#367c2b', '#76c043']
walmart_palette_d = ['#004c91', '#ffc220' , '#f47321', '#367c2b','#007dc6', '#76c043', '#78b9e7']
sns.set_palette(walmart_palette_d)

#Set parameters for graphs
plt.rc('figure', figsize=(13, 7))
# dictionary for font info
font = {'size'   : 14}
# pass in the font dict as kwargs
plt.rc('font', **font)

<hr style="border:3px solid black"> </hr>

## Overview

Don't you wish you could predict the future? With a little bit of data science we can get close. In this notebook our team walks through the full data science pipeline with some Walmart Retail sales data, and attempts to do just that. We acquire, clean, prepare, explore, model and evaluate the data in order to create a model that can predict sales for the next week. We will walk through how we created a linear regression model, using time series data, that was 29% more accurate than merely predicting that sales would be the same as they were the year before.

#### Data Intro
This data includes weekly sales information from 45 different walmart stores of various sizes (sqft). Indications of important holidays are notated for each week. There is also a CPI (Consumer Price Index), regional fuel price, and unemployment measurement, and regional average temperature for each observation. In addition to the historical data, which we thought would be a good indicator of price year over year, we also formed several hypotheses which we sought to answer over the course of this project.
   - Do Unemployment and CPI (inflation) have an effect on weekly sales? If so, would we need to lag them back? 
   - We think that the higher the CPI number, the lower the sales (because goods get more expensive when inflation goes up)
   - If Fuel Prices are higher will the sales nubmers be lower?
   - Does forecasted weather impact sales? i.e. next week’s weather, impact on sales. Higher temperatures, effect on sales? Lower temperatures, effect on sales?
   
#### Why is this important?
It's important to have an idea of what is to come in regards to sales for several reasons. For store managers and shareholders, having accurate predictions is beneficial for good inventory management and forecasting demand (impacts it can have on staffing and stocking). And a more efficient cost structure leads to more money going to the bottom line. Also, perhaps more importantly, if managers can accurately predict demand, customers will be more satisfied when they go to the store and see the products they want on the shelves. 

This type of technology can be adjusted to fit other retail businesses. A different store will have other factors that influence your sales? Economic factors? Interesting holiday spike? Located in a neighborhood with different shopping trends? That information can be captured to help create more accurate forecasts. 

# Acquire

The main table for this data came from [Kaggle](https://www.kaggle.com/aditya6196/retail-analysis-with-walmart-data), and the stores information table came directly from the original [Walmart Competition](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting). 

<hr style="border:2px solid blue"> </hr>

In [3]:
#Import data using acquire_data from new_wrangle.py
df= w.acquire_data()
#df_data = w.acquire_data()

FileNotFoundError: [Errno 2] No such file or directory: 'walmart_sales.csv'

In [ ]:
#take a look at the data
df.info()

In [ ]:
df.head()

**Key Findings and Takeaways**:
* Data sets provided in two .csv files that are merged into one dataframe for the project


<hr style="border:3px solid black"> </hr>

# Prepare

In this section of the pipeline, we needed to undertake the most difficult task for this project: Transforming time series data into data that is ready for a regression model. We had to capture the time data (numerically), historical sales and economic factor data, as well as the target (next week's sales) in one row. We also split the data into training and testing sets for exploration and modeling. 

<hr style="border:2px solid blue"> </hr>

In [ ]:
# Execute preparation function from new_wrangle.py
df= w.wrangle_walmart()

### Split and Scale Data

In [ ]:
#split and scale data using split_scale function in new_wrangle.py
train, test, X_train_scaled, X_test_scaled, y_train, y_test = w.split_scale(df, 'next_week_sales_target', scaler = MinMaxScaler())

In [ ]:
#take a look at new df after preperation was completed
df.info()

### Key Findings and Takeaways:
* Basic clean consisted of lowercasing features, renaming columns, converting selected dtypes appropriately, rounding decimals and converting date object into datetime 
* Identified a handful of outliers based on store type and weekly sales data and decided to reclassify store type in accordance with prevailing weekly sales threshold
* Feature engineering:
    * Added feature labeling holiday by Date: Super Bowl, Labor Day, Thanksgiving and Christmas
        * Additional column created with dummy variables for the holidays
        * `pre_christmas` feature was created to capture the Christmas holiday sales buildup two weeks prior to the holiday
    * Created additional feature containing the following week's 52 week (1yr) weekly sales lookback by shifting up weekly sales by 51 periods
    * Created monthly and quarterly rolling window features in each observation for this_week_unemployment, fuel_prices & CPI features
        * Additionally, created delta features for all above on a month-over-month and quarter-over-quarter basis
            * Each observation had the rolling window averages...look backs of 4 and 12 periods with percent change calculation
    * Created new index concating (current Date) + (store number) + (week+1 Date) features
    * **Target**: weekly sales shifted back 1 time period to be regressed on observation features from previous week

<hr style="border:3px solid black"> </hr>

# Explore

Here we take a look at the different features of our data set, to better understand the relationships between variables and our target.

<hr style="border:2px solid blue"> </hr>

## Univariate Exploration

In [ ]:
#explore our target
plt.figure(figsize=(8, 10))
sns.histplot(train,
                 x=train['next_week_sales_target'],
                 #hue='loan_status',
                 multiple='layer'
                 )
plt.title("Distribution of Next week sales")
plt.xlabel('Weekly sales in dollars')
plt.ylabel('Number of weeks')
plt.xticks(rotation=45)
plt.ticklabel_format(style = 'plain')

plt.show()

In [ ]:
# create temp df with a dateteime index for explore
df_data= w.acquire_data()
df_data1= w.acquire_data()
df_data['Date']= pd.to_datetime(df_data['Date'], dayfirst=True)
df_data = df_data.set_index('Date')

In [ ]:
# Annual weekly sales by month
plt.figure(figsize=(16, 10))
df_data.Weekly_Sales.groupby([df_data.index.year, df_data.index.month]).sum().unstack(0).plot(figsize = (14,10))
plt.title('Annual Weekly Sales by Month', fontsize = 16)
plt.ticklabel_format(style='plain', axis='y')
plt.show()

### Key Findings & Takeaways:
* 2012 appears to contradict the trends seen in the other years
    * After digging into the weekly sales, we discovered that this was due to a calendar effect where the end of weeks fell on different points at the end of March or beginning of April
        * Note for 2nd iteration, potential normalization 
***

In [ ]:
fig, ax = plt.subplots(2, 2)
fig.suptitle('Visualizing target vs features', fontsize = 15)
fig.tight_layout(h_pad = 2)
fig.set_figheight(15)
fig.set_figwidth(15)

plt.subplot(2, 2, 1)
ax[0, 0] = df_data.Weekly_Sales.resample('M').sum().plot()
ax[0, 0].set_title('Total Weekly Sales by Month')
plt.xticks(rotation=0)

plt.subplot(2, 2, 2)
ax[0, 1]= df_data.Unemployment.resample('M').mean().plot()
plt.xticks(rotation=0)
ax[0, 1].set_title('Average Unemployment by Month')

plt.subplot(2, 2, 3)
ax[1, 0]= df_data.Fuel_Price.resample('M').mean().plot()
ax[1, 0].set_title('Average Fuel Price by Month')
plt.xticks(rotation=0)

plt.subplot(2, 2, 4)
ax[1, 1]=  df_data.CPI.resample('M').mean().plot()
ax[1, 1].set_title('Average CPI by Month')
plt.xticks(rotation=0)

plt.show()

### Key Findings & Takeaways:

* Weekly sales spike significantly over the Nov-Dec months only to nosedive to annual lows in January
    * A rebound in sales during Feb after cyclical lows in Jan 
        * Much milder spikes appear consistent in the months of June and August
* Avg unemployment shows a clear declining trend whild avg fuel price shows a clear rising trend
    * Expectation is that this should influence a rising trend in weekly sales. something to be aware of
* Seasonality in both the avg deflated sales and avg temperature occurs over an annual basis
***

## Bivariate Exploration

In [ ]:
#visualize store_type by store_size
plt.figure(figsize=(14, 6))
sns.boxplot(x='store_type', y='store_size', data=train)
plt.title('Store Type by Size')

### Key Findings & Takeaways:
* Store type directly related to size
    * Anomalous observations were found in store types A & B appear more in line with store type C
        * These were: store 3, store 5, store 33, store 36 
        * Instead of dropping, we decided to reclassify these and include them in our type C data
***

In [ ]:
#visualize target with holidays
plt.figure(figsize=(14,6))
sns.boxplot(x= train['next_week_holiday_name'], y =train['next_week_sales_target'] )
plt.ticklabel_format(style='plain', axis='y')
plt.xlabel('\n Next Week - Holiday', fontsize = 13)
plt.ylabel('Next Week Sales', fontsize = 13)
plt.title('Sales by Holiday', fontsize = 16)
plt.show()

### Key Findings & Takeaways:
* Our Pre_Christmas & Thanksgiving sales appear to be strong sales drivers.  
    * Further investigation shows Thanksgiving sales are primarily driven by Black Friday specials
    * After noticing a run up in sales leading into the Christmas week, we created an additional feature to capture this period
        * The exploration above shows our pre_Christmas period being our strongest sales driver within these observations
            * The Christmas tag was applied to the week ending after the Christmas holiday negating the holiday impact; unlike the Thanksgiving holiday where Black Friday occurs in the same week
***

In [ ]:
#correlation with target and all other features
wal_corr = train.corr()['next_week_sales_target'].sort_values(ascending=False)
#visualize the correlation between features and our target (next week sales)
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(train.corr()[['next_week_sales_target']].sort_values(by='next_week_sales_target', ascending=False), vmin=-1, vmax=1, annot=True, cmap='mako_r')
heatmap.set_title('Features Correlating with weekly sales', fontdict={'fontsize':18}, pad=16);

### Key Findings & Takeaways:
* Visualizing the correlation between our target and other features allowed us to see:
    * Highest correlation between our target and last years sales, this weeks store size, and holiday indicators.    
***

In [ ]:
df_data1.head(1)


## Multivariate Exploration

In [ ]:
# create column list to look at for pairplot
cols = ['Weekly_Sales','Temperature','Fuel_Price','CPI',
        'Unemployment','Type','Size']

In [ ]:
# create pairplot
sns.pairplot(data = df_data[cols], hue = 'Type', corner=True)

### Key Findings & Takeaways

* No trends really jump out right away. Biggest thing, is that when unemployment is higher, sales are lower. Maybe people are out of jobs and they're not shopping. Also it doesn't appear that there was much time where the unemployment *was* high during this time period here. (between 2010 - 2012)
* There is some gap in in CPI between 150 and 180 or so. Maybe there was some sort of jump in the market either up or down. 
* Highest sales for the big stores (in blue), were when the temperature was in the 40s. 

In [ ]:
# set up dataframe to use in lag plots
# columns we'll need
cols = ['next_week_sales_target', 'next_week_holiday_name', 'next_week_date']

# create temp dataframe with columns, set to datetime index, sort the index and drop nulls
df_data1 = df[cols].set_index('next_week_date').sort_index().dropna()

In [ ]:
# Various Target Lag plots

fig, ax = plt.subplots(2, 2)
fig.suptitle('Weekly sales vs various lags')
fig.tight_layout(h_pad = 2, w_pad = 4)
fig.set_figheight(18)
fig.set_figwidth(15)


plt.subplot(2, 2, 1)
ax1 = sns.scatterplot(df_data1.groupby('next_week_date').next_week_sales_target.agg('sum'), 
                      df_data1.groupby('next_week_date').next_week_sales_target.agg('sum').shift(-1), 
                      hue=df_data1.next_week_holiday_name)
plt.xlabel('$weekly sales$')
plt.ylabel('$sales - 1$')
ax1.ticklabel_format(style = 'plain')
ax1.set_title('Lag plot with lag=1')

plt.subplot(2, 2, 2)
ax2 = sns.scatterplot(df_data1.groupby('next_week_date').next_week_sales_target.agg('sum'), 
                      df_data1.groupby('next_week_date').next_week_sales_target.agg('sum').shift(-4), 
                      hue=df_data1.next_week_holiday_name)
plt.xlabel('$weekly sales$')
plt.ylabel('$sales - 4$')
ax2.ticklabel_format(style = 'plain')
ax2.set_title('Lag plot with lag=4')

plt.subplot(2, 2, 3)
ax3= sns.scatterplot(df_data1.groupby('next_week_date').next_week_sales_target.agg('sum'), 
                      df_data1.groupby('next_week_date').next_week_sales_target.agg('sum').shift(-12), 
                      hue=df_data1.next_week_holiday_name)
plt.xlabel('$weekly sales$')
plt.ylabel('$sales - 12$')
ax3.ticklabel_format(style = 'plain')
ax3.set_title('Lag plot with lag=12')

plt.subplot(2, 2, 4)
ax4= sns.scatterplot(df_data1.groupby('next_week_date').next_week_sales_target.agg('sum'), 
                      df_data1.groupby('next_week_date').next_week_sales_target.agg('sum').shift(-52), 
                      hue=df_data1.next_week_holiday_name)
plt.xlabel('$weekly sales$')
plt.ylabel('$sales - 52$')
ax4.ticklabel_format(style = 'plain')
ax4.set_title('Lag plot with lag=52')

plt.show()


### Key Findings & Takeaways:
* There is a clear linear correlation when lagging the data on an annual basis (52 weeks)
* There also appears to be significant correlation when lagging one period
    * outliers in the one period lag appear evenly distributed across store size groupings and appear to mirror each other suggesting the same time periods may be mirrored
        * perhaps these are due to the holiday periods; check to see if any possibility of addressing
            * CONFIRMED: the outliers occur in consecutive dates surrounding the Thanksgiving and Christmas holidays.  Specifically, [2010-11-19, 2010-11-26, 2010-12-17, 2010-12-24, 2011-11-18, 2011-11-25, 2011-12-16, 2011-12-23]
***

## Hypothesis Testing

### Hypothesis 1: Pearson's (cont vs cont)
$H_0$: There is no correlation between next week sales and this week sales 

$H_a$: There is a correlation between next week sales and this week sales

In [ ]:
#pearsons correlation on entire train set
#number of rows
n = train.shape[0] 

#degrees of freedom- how much the data can vary
deg_f = n-2 

#confidence interval (!)
conf_in = 0.95

#calculate alpha
alpha = 1- conf_in

#identify x and y
x= train.next_week_sales_target
y= train.this_week_sales

In [ ]:
#calculate r and p values
r, p = stats.pearsonr(x,y)
r,p

In [ ]:
print(f'We calculate a pearson r of {r:3f} and a statistical certainty p of {p:4f}')
print(f'Because p {p:4f} < α  {alpha:4f}, we can reject our null hypothesis')

### Hypothesis 2: Pearson's (cont vs cont)
$H_0$: There is no correlation between next week's sales and sales from the same period last year

$H_a$: There is a correlation between next week's sales and sales from the same period last year

In [ ]:
#pearsons correlation on entire train set
#number of rows
n = train.shape[0] 

#degrees of freedom- how much the data can vary
deg_f = n-2 

#confidence interval (!)
conf_in = 0.95

#calculate alpha
alpha = 1- conf_in

In [ ]:
#identify x and y
x= train.next_week_sales_target
y= train.next_week_1_year_ago

In [ ]:
#calculate r and p values
r, p = stats.pearsonr(x,y)
r,p

In [ ]:
print(f'We calculate a pearson r of {r:3f} and a statistical certainty p of {p:4f}')
print(f'Because p {p:4f} < α  {alpha:4f}, we can reject our null hypothesis')

## Hypothesis 3: T-Test (discrete vs cont)
$H_0$: There is no relationship between this next_weeks_sales_target and pre_christmas

$H_a$: There is a relationship between this next_weeks_sales_target and pre_christmas

In [ ]:
#confidence interval (!)
conf_in = 0.95

#calculate alpha
alpha = 1- conf_in

In [ ]:
#check sample size, must be more than 30 to meet assumptions
train.next_week_sales_target.shape, train.pre_christmas.shape

In [ ]:
#check variance
train.next_week_sales_target.var(), train.pre_christmas.var()

In [ ]:
#2 tailed t-test
t, p = stats.ttest_ind(train.next_week_sales_target,train.pre_christmas, equal_var=False)
t, p

In [ ]:
print(f'We calculate a t of {t:4f} and a statistical certainty p of {p:4f}')
print(f'Because t {t:4f} < α  {alpha:2f}, we can reject our null hypothesis')

### Key Findings and Takeaways:
* These were all must haves.  
    * If weekly sales observations were independent, there would be no common drivers to explore and build a regression model around that would enable us to forecast weekly sales one week in advance.  
    * Additionally, establishing statistical significance in the relationship between our target and our pre_christmas feature enables us to model what our exploration clearly defined as our most significant source of seasonality
    
Given that the p values were all below 0.05, we have a project!
***

## Addressing Collinearity in our Engineered Features

Some features that we created (the changes in gas prices and changes in CPI and such) probably were correlated to each other and not just to the target. This will present an issue when going into the modeling phase. We need features to correlate with the target, not with each other. That's called *Multicolinearity* and we deal with it here. 

### - Testing for collinearity: engineered unemployment features

In [ ]:
w.collinearity_unemployment(X_train_scaled)

#### Findings & Takeaways: unemployment
* `this_week_unemployment` nearly perfectly correlated with rolling avgs; p value under 0.05, recommend dropping unemployment
* Rolling averages nearly perfectly correlated with each other; p value under 0.05, recommend dropping `quarterlyrolling`
* Deltas correlation co-efficient 0.42. p value under 0.05; is this correlation significant enough to drop?
***

### - Testing for collinearity: engineered fuel price features

In [ ]:
w.collinearity_fuel(X_train_scaled)

#### Findings & Takeaways: fuel price
* `fuel_price` nearly perfectly correlated with rolling avgs; p value under 0.05, recommend dropping `fuel_price`
* 4wk rolling has a 0.37 corr coefficient vs avgQoQ; p value under 0.05, is this threshold high enough to drop?
* Rolling averages strongly correlated with each other; p value under 0.05, recommend dropping quarterly rolling as 4wk captures both quarterly rolling and fuel price well
* Deltas correlation co-efficient 0.2; p value under 0.05, recommend keeping both
***

### - Testing for collinearity: engineered CPI features

In [ ]:
w.collinearity_cpi(X_train_scaled)

#### Findings & Takeaways: CPI
* `CPI` nearly perfectly correlated with rolling avgs; p value under 0.05, recommend dropping `CPI`
* Rolling averages nearly perfectly correlated with each other; p value under 0.05, recommend dropping quarterly rolling
* Deltas correlation co-efficient 0.25. p value under 0.05; is this correlation significant enough to drop?
***

#### - Iterative Feature Elimination due to Auto Correlation findings

In [ ]:
#features to drop before modeling due to autocorrelation
col = ['this_week_unemployment', 'fuel_price', 'CPI', 'unemp_quarterly_rolling', 'fuel_quarterly_rolling', 'cpi_quarterly_rolling']

#drop columns that have coliniarity
X_train_scaled = X_train_scaled.drop(columns = col)
X_test_scaled = X_test_scaled.drop(columns = col)

<hr style="border:3px solid black"> </hr>

# Modeling

In this phase we create a regression model using the following steps.
1. Establish a baseline to test our model against
2. Create and fit various types of regression models
3. Evaluate using Cross validation
4. Test the best model on unseen data 

<hr style="border:2px solid blue"> </hr>

### Summary Notes for Modeling Section:
    
* **Scaler utilized** = MinMax()

* **Baselines**: 1) weekly sales mean & 2) 52 week look back to previous year weekly sales ('next_week_1_year_ago')

* **Algorithms utilized:**
    - Linear Regression(OLS)
    - Lasso Lars
    - TweedieRegressor (GLM)
    - Polynomial Regression

#### For each model we used:

 - **GridSearchCV**: grid search cross-validation (GridSearchCV) for hyperparameter optimization.
 
- **Feature Engineering:**

   - **RFE :** (recursive feature elimination)  recursively removes attributes to meet the required number of features and then builds a model on those attributes that remain.
   - **Select KBest:** removes all but the highest scoring features.
***

### Features

In [ ]:
# set model features
features = ['store_size', 'this_week_unemployment', 'next_week_1_year_ago', 'this_week_sales', 'pre_christmas']

### Y_train and Y_test

In [ ]:
# convert y_train y _ validate to df
y_train_df = pd.DataFrame( {'actual': y_train})
y_test_df = pd.DataFrame( {'actual': y_test})

### Baseline 1: mean of weekly sales

In [ ]:
# create the baseline using mean of actual weekly sales
baseline= y_train_df['actual'].mean()

In [ ]:
# create column called baseline to compare
y_train_df['baseline'] = baseline

In [ ]:
# calculate RMSE for baseline model
rmse_baseline_train= math.sqrt(mean_squared_error(y_train_df.actual, y_train_df.baseline))

In [ ]:
# create a dataframe to make data easier to visualize/understand
metric_df = pd.DataFrame(data=[{
    'model': "Baseline (using mean)",
    'rmse_train': round(rmse_baseline_train, 4),
    'r^2': round(explained_variance_score(y_train_df.actual, y_train_df.baseline), 4)

}])

round(metric_df, 4)

### Baseline 2 : prior year's weekly sales

In [ ]:
# baseline using last years sales as predictions
y_train_df['last_year_baseline'] = train['next_week_1_year_ago']

In [ ]:
# calculate RMSE for baseline 2 model
rmse_baseline2_train= math.sqrt(mean_squared_error(y_train_df.actual, y_train_df.last_year_baseline))

In [ ]:
# RMSE Baseline 2
round(rmse_baseline2_train, 4)

In [ ]:
# add Baseline 2 to our metric_df
metric_df = metric_df.append(
    {
    'model': "Baseline (using last year's sales)",
    'rmse_train': round(rmse_baseline2_train, 2),
    'r^2': round(explained_variance_score(y_train_df.actual, y_train_df.last_year_baseline),4),
    }, ignore_index=True)

round((metric_df), 4)

## MODEL 1: LinearRegression (OLS)

### - Hyperparameter optimization with Gridsearch

In [ ]:
# specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {'normalize': [ True, False],
          'fit_intercept': [True, False]}

In [ ]:
# deploy gridsearch function using 
round(m.gridsearch(X_train_scaled, y_train_df, LinearRegression() , params, 'neg_root_mean_squared_error'), 4)

### - Feature Selection using Select K Best

In [ ]:
# look at columns in y_train
y_train_df.columns

In [ ]:
# using my function for SelectkBest
top_sb = m.select_kbest(X_train_scaled, y_train, 10)

In [ ]:
# create the model and calculate RMSE
ols_sb = m.create_model(X_train_scaled[top_sb], y_train_df, 'actual',\
                       LinearRegression(normalize=False,\
                        fit_intercept=True ), 'modelOLS' )
round(ols_sb['rmse'], 4)

### - cross validation

In [ ]:
# create the model
clf = LinearRegression(normalize=True, fit_intercept=True )
# cv = number of folds
cross_val_score(clf, X_train_scaled[top_sb], y_train, cv=3, scoring = 'neg_root_mean_squared_error')

In [ ]:
# add the ols_sb model metrics to metric_df
metric_df = metric_df.append(
    {
    'model': 'ols_sb',
    'rmse_train': ols_sb['rmse'],    
    'r^2' : ols_sb['r2']}, ignore_index=True)

### - Feature Selection using  RFE

In [ ]:
# use the function to get RFE selected features
top_rfe = m.select_rfe(X_train_scaled, y_train, 10,LinearRegression(normalize=True, fit_intercept=True ))

In [ ]:
# create the model and calculate RMSE
ols_rfe = m.create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LinearRegression(normalize=True, fit_intercept=True ), 'modelOLS' )
round(ols_rfe['rmse'], 4)

### - cross validation

In [ ]:
# create the model
clf = LinearRegression(normalize=True, fit_intercept=True )
# cv = number of folds
cross_val_score(clf, X_train_scaled[top_rfe], y_train, cv=3, scoring = 'neg_root_mean_squared_error')

In [ ]:
# add the ols_rfe model metrics to metric_df
metric_df = metric_df.append(
    {
    'model': 'ols_rfe',
    'rmse_train': ols_rfe['rmse'],    
    'r^2' : ols_rfe['r2']}, ignore_index=True)

## MODEL 2: LassoLars

### - Hyperparameter optimization with Gridsearch

In [ ]:
# specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'normalize': [True, False],
          'fit_intercept':[True, False],
           'alpha': [1.0, 0]
         }

In [ ]:
# use the function to get the combinations of parameters
round(m.gridsearch (X_train_scaled, y_train, LassoLars() , params, 'neg_root_mean_squared_error'), 4)

### - Feature Selection using Select K Best

In [ ]:
# using my function for SelectkBest
top_sb =m.select_kbest(X_train_scaled, y_train, 9)

In [ ]:
# create the model and calculate RMSE
lasso_skb = m.create_model(X_train_scaled[top_sb], y_train_df, 'actual', LassoLars(alpha = 1, normalize= True, fit_intercept= True), 'modelLasso' )
round(lasso_skb['rmse'], 2)

### - cross validation

In [ ]:
# create the model
clf = LassoLars(alpha = 1, normalize= False, fit_intercept= True)
# cv = number of folds
cross= cross_val_score(clf, X_train_scaled[top_sb], y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

In [ ]:
# add the lasso_skb model metrics to metric_df
metric_df = metric_df.append(
    {
    'model': 'lasso_skb',
    'rmse_train': lasso_skb['rmse'],    
    'r^2' : lasso_skb['r2']}, ignore_index=True)

### - Feature Selection using  RFE

In [ ]:
# get the top 8 features
top_rfe = m.select_rfe(X_train_scaled, y_train, 6, LassoLars(alpha = 1, normalize= True, fit_intercept= True) )

In [ ]:
# use the function to get the combinations of parameters
round(m.gridsearch (X_train_scaled, y_train, LassoLars() , params, 'neg_root_mean_squared_error'), 4)

In [ ]:
# create the model and calculate RMSE
lasso_rfe = m.create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LassoLars(alpha = 1, normalize= False, fit_intercept= True), 'modelLasso' )
round(lasso_rfe['rmse'], 4)

### - cross validation

In [ ]:
# create the model
clf = LassoLars(alpha = 1, normalize= False, fit_intercept= True)
# cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_rfe], y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

In [ ]:
# add the lasso_rfe model to metric_df
metric_df = metric_df.append(
    {
    'model': 'lasso_rfe',
    'rmse_train': lasso_rfe['rmse'],    
    'r^2' : lasso_rfe['r2']}, ignore_index=True)

## MODEL 3: TweedieRegressor (GLM)

### - Hyperparameter optimization with Gridsearch

In [ ]:
# specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'power': [0.0, 1],
           'fit_intercept' : [True , False],
          'warm_start': [True, False], 
           'alpha': [1.0, 0.0]
         }
# use the function
round(m.gridsearch (X_train_scaled, y_train,TweedieRegressor() , params, 'neg_root_mean_squared_error'), 4)

### - Feature Selection using Select K Best

In [ ]:
# using my function for SelectkBest
top_sb =m.select_kbest(X_train_scaled, y_train, 8)

In [ ]:
# create the model and calculate RMSE
gml_skb = m.create_model(X_train_scaled[top_sb], y_train_df, 'actual',TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False), 'modelgml' )
round(gml_skb['rmse'], 4)

### - cross validation

In [ ]:
# create the model
clf = TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,warm_start= False)
# cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_sb], y_train, scoring = 'neg_root_mean_squared_error',cv=3)
cross

In [ ]:
# add the gml_skb model to metric_df

metric_df = metric_df.append(
    {
    'model': 'gml_skb',
    'rmse_train': gml_skb['rmse'],    
    'r^2' : gml_skb['r2']}, ignore_index=True)

### -  Feature Selection using  RFE

In [ ]:
# use function to get the top 8 features
top_rfe = m.select_rfe(X_train_scaled, y_train, 7, TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False) )

In [ ]:
# create the model and calculate RMSE
gml_rfe = m.create_model(X_train_scaled[top_rfe], y_train_df, 'actual',TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False), 'modelgml' )
round(gml_rfe['rmse'], 4)

### - cross validation

In [ ]:
# create the model
clf = TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,warm_start= False)
# cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_rfe], y_train, scoring = 'neg_root_mean_squared_error', cv=3)
cross

In [ ]:
# add the gml_rfe model to metric_df
metric_df = metric_df.append(
    {
    'model': 'gml_rfe',
    'rmse_train': gml_rfe['rmse'],    
    'r^2' : gml_rfe['r2']}, ignore_index=True)

## MODEL 4: Polynomial Regression

### - Feature Selection using  Select K Best

In [ ]:
# use function to get the top 8 features
top_sb =m.select_kbest(X_train_scaled, y_train, 10)

In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree = 2) 

# fit and transform X_train_scaled
X_train_degree3 = pf.fit_transform(X_train_scaled[top_sb])

# transform X_validate_scaled & X_test_scaled
X_test_degree3 = pf.transform(X_test_scaled[top_sb])

### - Hyperparameter optimization with Gridsearch

In [ ]:
# specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'normalize': [True, False],
          'fit_intercept':[True, False],
         }

round(m.gridsearch (X_train_degree3, y_train, LinearRegression() , params, 'neg_root_mean_squared_error'), 4)

In [ ]:
# create the model and calculate RMSE
pol_skb = m.create_model(X_train_degree3, y_train_df, 'actual',LinearRegression( normalize=True, fit_intercept = True ), 'pol2_skb' )
round(pol_skb['rmse'], 4)

### - cross validation

In [ ]:
# create the model
clf = LinearRegression( normalize=False, fit_intercept = True )
# cv = number of folds
cross = cross_val_score(clf, X_train_degree3, y_train, cv=5, scoring = 'neg_root_mean_squared_error')
cross

In [ ]:
# add the pol_skb model to metric_df
metric_df = metric_df.append(
    {
    'model': 'pol_skb',
    'rmse_train': pol_skb['rmse'],    
    'r^2' : pol_skb['r2']}, ignore_index=True)

### - Feature Selection using  RFE

In [ ]:
# get the top 8 features by RFE
top_rfe_pol = m.select_rfe(X_train_scaled, y_train, 11, LinearRegression(normalize=False, fit_intercept = False) )

In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree = 2) 

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train_scaled[top_rfe_pol])

# transform X_validate_scaled & X_test_scaled
X_test_degree2 = pf.transform(X_test_scaled[top_rfe_pol])

### - Hyperparameter optimization with Gridsearch

In [ ]:
round(m.gridsearch (X_train_degree2, y_train, LinearRegression() , params, 'neg_root_mean_squared_error'), 4)

In [ ]:
# create the model and calculate RMSE
pol_rfe = m.create_model(X_train_degree2, y_train_df, 'actual',LinearRegression( normalize=False, fit_intercept = False ), 'pol3_RFE' )
round(pol_rfe['rmse'], 4)

### - cross validation

In [ ]:
# create the model
clf = LinearRegression( normalize=False, fit_intercept = False )
# cv = number of folds
cross = cross_val_score(clf, X_train_degree2, y_train, cv=5, scoring = 'neg_root_mean_squared_error')
cross

In [ ]:
# add the pol_rfe model to metric_df
metric_df = metric_df.append(
    {
    'model': 'pol_rfe',
    'rmse_train': pol_rfe['rmse'],    
    'r^2' : pol_rfe['r2']}, ignore_index=True)

## Modeling Results

In [ ]:
round(metric_df.sort_values('rmse_train'), 4)

In [ ]:
print('The best model is' )
round(metric_df.nsmallest(1, 'rmse_train'), 4)

### Key Findings and Takeaways**

* Our top performing model is the Polynomial Regression 2nd Degree model utilizing the Recursive Feature Elimnation (RFE) feature selector
    * According this this model, the top 11 selected features based on the the RFE class class are: ['this_week_sales', 'temperature', 'store_size', 'next_week_1_year_ago', 'christmas', 'pre_christmas', 'super_bowl', 'thanksgiving', 'avgMoM_perc_cpi', 'avgMoM_perc_unemp', 'avgQoQ_perc_unemp']
    * Note: feature numbers varied by model and were chosen via an iterative process that sought out the best performing scenario
***

## **Test**

In [ ]:
#use function to create and calculate the metrics
pol_reg_test =  m.create_model(X_test_degree2, 
                              y_test_df, 'actual',LinearRegression( normalize=False, fit_intercept = True ), 'test_polreg' )

round(pol_reg_test['rmse'], 4)

In [ ]:
print('The Baseline RMSE is $', round(rmse_baseline2_train,4))
print('The Best Model RMSE on unseen data is $', round(pol_reg_test['rmse'],4))

<div class="alert alert-block alert-success">
<b>Key Findings and Takeaways:</b> Our Polynomial Regression model utilizing the RFE selector performed very well on the Test data set and showed no signs of being overfit. The difference in the Train and Test RMSE scores was roughly 1.7%.  Additionally, based on the RMSE scores, our model Test outperformed the Baseline by roughly 29%.    
</div>

## **Visual Evaluation**

In [ ]:
# Calculating R2 for our model predictions
evs = explained_variance_score(y_train_df.actual, y_train_df.pol3_RFE)
print('Explained Variance = ', round(evs,3))

# Calculating R2 for our Baseline 2
evs = explained_variance_score(y_train_df.actual, y_train_df.last_year_baseline)
print('Explained Variance = ', round(evs,3))

In [ ]:
plt.figure(figsize=(16,8))
plt.scatter(y_train_df.actual, y_train_df.last_year_baseline, alpha=.8, color="red", label='baseline')
plt.annotate("Baseline: Predict Using Mean", (16, 9.5))
plt.plot(y_train_df.actual, y_train_df.actual, alpha=.5, color="blue", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5, 3.5), rotation=15.5)

plt.scatter(y_train_df.actual, y_train_df.modelgml, 
            alpha=.5, color="green", s=100, label="Model: Tweedie Regressor")
plt.scatter(y_train_df.actual, y_train_df.modelOLS, 
            alpha=.5, color="yellow", s=100, label="Model: OLS")
plt.scatter(y_train_df.actual, y_train_df.pol3_RFE, 
            alpha=.4, color="purple", s=100, label="Model: Polynomial RFE")


plt.legend()
plt.xlabel("Actual Weekly Sales")
plt.ylabel("Predicted Weekly Sales")
plt.title("Model Evaluation: Model Predictions vs Actual Sales")
plt.ticklabel_format(style = 'plain')
plt.show()

### Key Findings and Takeaways**

* Based on our R2 scores, our baseline was a fairly predictive methodology and left a small window for improvement  
    * 97.4% of the variance in our target could be explained by last year's sales data alone
    * red dots in the chart above show how tight last year's sales data lies to the actual sales data our model was attempting to predict
* Nevertheless, we were able to use the provided data, engineer a few features and successfully build a model that improved on the baseline and explains 98.7% of the target variance.  

***

## -  Polynomial Features

In [ ]:
# Best model
pf = PolynomialFeatures(degree = 2)
pf.fit_transform(X_train_scaled[top_rfe_pol])
mod =LinearRegression( normalize=False, fit_intercept = False )
mod.fit(X_train_degree2, y_train_df['actual'])
X_train_top = X_train_scaled[top_rfe_pol]


### -Get Features Names

In [ ]:

#create a df to store the features 
feature_importances = pd.DataFrame(mod.coef_, 
                                   index = pf.get_feature_names(X_train_top.columns), 
                                   columns=['importance']).sort_values('importance', ascending=False)

#reset index
feature_importances = feature_importances.reset_index().rename( columns ={'index': 'features'} )

# Index should start at 1 so it can show the rank of each feature
feature_importances.index = feature_importances.index + 1

In [ ]:
#create a new df, that have the top 5 positive and negative features

In [ ]:
 df_top =pd.concat([feature_importances.head(10), feature_importances.tail(10)])

In [ ]:
plt.figure(figsize=(10,10))
clrs = ['yellow' if (x < 0) else 'blue' for x in df_top.importance ]
sns.barplot(x='importance', y = 'features', data = df_top , palette=clrs)
plt.ylabel('')
plt.yticks(size=16)
plt.xticks(rotation=45)
plt.title('Most Important Polynomial Features', fontsize = 18)


plt.show()

## Conclusion 

We successfully created a model that predicts next weeks sales better than the baseline by 29%. Showing that there is value in the external factors such as inflation, and unemployment, sales from the current week. 

With more time we would like to scale this model up or down. Perhaps clustering stores together to find similar economic regions. We would cluster stores together not just by size, but by similar changes in fuel prices, or unemployment numbers. This type of analysis could be of help to regional managers. Or perhaps we gather data by Walmart departments. While economic factors might have one effect on the stores as a whole, how do they affect different departments? Is the grocery department affected the same way as the electronics department?  


Thank you for taking the time to read this notebook. 

<img src="https://i5.walmartimages.com/asr/e9f4a121-27fc-4226-b059-309ddd73615b_1.d55cf65a92feb2eacea4433c80d940c6.jpeg?odnWidth=1000&odnHeight=1000&odnBg=ffffff" alt="Smile" width="600"/>
